## First Round Сountry Сheck

used "google_domain": "google.com"

[] - iOS, [] - Android
-----------------------
- [-],[-] Austria
- [-],[-] Belgium
- [-],[+] Croatia
- [+],[-] Czechia (Czech Republic)
- [+],[+] Denmark
- [+],[+] Estonia
- [+],[-] Finland
- [-],[-] France
- [-],[-] Germany
- [-],[+] Greece
- [+],[-] Hungary
- [-],[-] Italy
- [+],[-] Latvia
- [-],[+] Lithuania
- [-],[-] Luxembourg
- [+],[-] Malta
- [+],[+] Netherlands
- [-],[+] Poland
- [+],[-] Portugal
- [-],[-] Slovakia
- [+],[+] Slovenia
- [-],[+] Spain
- [-],[-] Sweden
- [-],[+] Bulgaria (from March 2024)
- [+],[-] Romania (from March 2024)

Non-EU Member States:
---
- [-],[-] Iceland
- [+],[-] Liechtenstein
- [-],[-] Norway
- [-],[+] Switzerland

Outside Schengen but with special status:
---
- [+],[+] Ireland 
- [+],[+] United Kingdom
- [-],[-] Cyprus 

North American countries:
---
- [+],[+] Canada
- [+],[+] United States
- [-],[+] Mexico

In [6]:
from serpapi import GoogleSearch
from dotenv import load_dotenv
import pandas as pd
import time
import json
import os

load_dotenv()
api_key = os.getenv("API_KEY")

In [7]:
path = 'location_domain_table.csv'
df = pd.read_csv(path)
df.head()

,location,google_domain,gl(not needed),Region,EU member,Schengen Agreement
0,Austria,google.at,at,Europe,True,True
1,Belgium,google.be,be,Europe,True,True
2,Bulgaria,google.bg,bg,Europe,True,True
3,Canada,google.ca,ca,Northern America,False,False
4,Croatia,google.hr,hr,Europe,True,True


In [90]:
search = GoogleSearch({
    "engine": "google_jobs",
    "q": "Android developer", 
    "location": "Mexico",
    "api_key": api_key
  })
result = search.get_dict()

file_path = r"Test\Mexico1.json"  

# Saving JSON to a file
with open(file_path, "w", encoding="utf-8") as file:
    json.dump(result, file, ensure_ascii=False, indent=4)

Version 1:
search = GoogleSearch({
    "engine": "google_jobs",
    "q": "Job title", 
    "location": "Country",
    "api_key": api_key
  })

## New version

## Move files to their folders

In [91]:
import shutil

# Путь к папке "Data Collection/google_domain_dot_com"
base_dir = os.path.join(os.getcwd(), "google_domain_dot_com")
android_dir = os.path.join(base_dir, "Android")
ios_dir = os.path.join(base_dir, "iOS")

# Убедимся, что папки Android и iOS существуют
os.makedirs(android_dir, exist_ok=True)
os.makedirs(ios_dir, exist_ok=True)

# Перебираем файлы в папке google_domain_dot_com
for file_name in os.listdir(base_dir):
    # Полный путь к файлу
    file_path = os.path.join(base_dir, file_name)

    # Проверяем, является ли это файлом (а не папкой)
    if os.path.isfile(file_path):
        # Если файл соответствует шаблону для iOS
        if file_name.endswith(".json") and not file_name[:-5].endswith("1"):
            shutil.move(file_path, os.path.join(ios_dir, file_name))
        # Если файл соответствует шаблону для Android
        elif file_name.endswith(".json") and file_name[:-5].endswith("1"):
            shutil.move(file_path, os.path.join(android_dir, file_name))

print("Файлы успешно перемещены по папкам!")


Файлы успешно перемещены по папкам!


## Make markdown file with the results

In [92]:
import os
import json

# Путь к основной папке
base_path = "google_domain_dot_com"
android_path = os.path.join(base_path, "Android")
ios_path = os.path.join(base_path, "iOS")
markdown_path = os.path.join(base_path, "results.md")

# Проверка файла на "удачный" или "пустой"
def is_valid_file(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
            if data.get("search_information", {}).get("jobs_results_state") == "Fully empty" and \
               data.get("error") == "Google hasn't returned any results for this query.":
                return False
            return True
    except Exception as e:
        print(f"Ошибка при обработке {file_path}: {e}")
        return False

# Сканирование папок и сбор данных
results = {}

for folder, platform in [(ios_path, "iOS"), (android_path, "Android")]:
    for file_name in os.listdir(folder):
        if file_name.endswith(".json"):
            file_path = os.path.join(folder, file_name)
            country = file_name.replace(".json", "").replace("1", "").strip()
            is_android = "1" in file_name
            platform_key = "Android" if is_android else "iOS"
            is_valid = is_valid_file(file_path)
            
            if country not in results:
                results[country] = {"iOS": "-", "Android": "-"}
            
            results[country][platform_key] = "+" if is_valid else "-"

# Создание Markdown-файла
with open(markdown_path, "w", encoding="utf-8") as md_file:
    md_file.write("[] - iOS, [] - Android\n")
    md_file.write("-----------------------\n")
    for country, platforms in sorted(results.items()):
        ios_status = platforms["iOS"]
        android_status = platforms["Android"]
        md_file.write(f"- [{ios_status}],[{android_status}] {country}\n")

print(f"Markdown файл создан: {markdown_path}")


Markdown файл создан: google_domain_dot_com\results.md
